In [1]:
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import pandas as pd

import sys
sys.path.append('../')
from src import *

random.seed(1234)
np.random.seed(1234)

In [2]:
PATH_DATA = '/home/disij/projects/acdc/data/'
OUTPUT_DIR = "/extra/disij0/data/flow_cytometry/flowMP_output/"
PATH_SAMPLES = OUTPUT_DIR + "BMMC_accepted_samples"
FILENAME_PREDICTIONS = OUTPUT_DIR + "BMMC_predictions.csv.gz"

Load BMMC dataset from [ACDC paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5447237/pdf/btx054.pdf)...

In [3]:
### LOAD DATA ###
path = PATH_DATA + 'BMMC_benchmark/'
df = pd.read_csv( path + 'BMMC_benchmark.csv.gz', sep=',', header = 0, \
                 compression = 'gzip', engine='python')
table = pd.read_csv(path + 'BMMC_table.csv', sep=',', header=0, index_col=0)


channels = ['CD45','CD45RA', 'CD19', 'CD11b', 'CD4', 'CD8', 'CD34',
           'CD20', 'CD33', 'CD123', 'CD38', 'CD90', 'CD3']
df.columns = channels + ['cell_type']
df = df[df.cell_type != 'NotGated']

### five cell types below are the ones that we do not have prior information about.
### in acdc implementation, they are all catagorized as "unknown", yet since we are not able
### to handle unknown cell types, we remove all instances of these types
### proportion of "unknown" is 24.49% in total
df = df.loc[df['cell_type'] != 'Megakaryocyte']
df = df.loc[df['cell_type'] != 'CD11bmid Monocyte']
df = df.loc[df['cell_type'] != 'Platelet']
df = df.loc[df['cell_type'] != 'Myelocyte']
df = df.loc[df['cell_type'] != 'Erythroblast']

table = table.fillna(0)
X = df[channels].values

### transform data
data = np.arcsinh((X-1.)/5.)
N, d = data.shape
emp_bounds = np.array([[data[:,d].min(), data[:,d].max()] for d in range(data.shape[1])])
ct2idx = {x:i for i,x in enumerate(table.index)}
idx2ct = [key for idx, key in enumerate(table.index)]
Y = np.array([ct2idx[_] for _ in df.cell_type])

Learn MP trees and write accepted samples to file...

In [4]:
%%time


###################### Parallel run #####################
# n_mcmc_chain = 50
# n_mcmc_samples = 3000
# chains = range(n_mcmc_chain)
# num_cores = multiprocessing.cpu_count()
# accepted_MP = Parallel(n_jobs=num_cores)(delayed(MP_mcmc)\
#                 (data, emp_bounds, table, i, n_mcmc_samples) for i in chains)
# write_chains_to_file(accepted_MP, PATH_SAMPLES)



### Here we run sequentially to monitor the effect of ensembling multiple chains
n_mcmc_chain = 50
n_mcmc_samples = 3000
accepted_MP = []
for i in range(n_mcmc_chain):
    print "Sampling Chain %d..." % i
    accepted_MP.append(MP_mcmc(data, emp_bounds, table, i, n_mcmc_samples))
    burnt_samples = [sample for chain in accepted_MP[-1:] for sample in chain[-20:]]   
    Y_predict = classify_cells_majority(data, burnt_samples, table, ct2idx)
    accuracy = sum(Y == Y_predict)*1.0/ N
    print "Accuracy of cell classification on all data: %.3f" % (accuracy)

write_chains_to_file(accepted_MP, PATH_SAMPLES)

Sampling Chain 0...
Accuracy of cell classification on all data: 0.897
Sampling Chain 1...
Accuracy of cell classification on all data: 0.842
Sampling Chain 2...
Accuracy of cell classification on all data: 0.793
Sampling Chain 3...
Accuracy of cell classification on all data: 0.832
Sampling Chain 4...
Accuracy of cell classification on all data: 0.775
Sampling Chain 5...
Accuracy of cell classification on all data: 0.774
Sampling Chain 6...
Accuracy of cell classification on all data: 0.798
Sampling Chain 7...
Accuracy of cell classification on all data: 0.657
Sampling Chain 8...
Accuracy of cell classification on all data: 0.844
Sampling Chain 9...
Accuracy of cell classification on all data: 0.752
Sampling Chain 10...
Accuracy of cell classification on all data: 0.825
Sampling Chain 11...
Accuracy of cell classification on all data: 0.880
Sampling Chain 12...
Accuracy of cell classification on all data: 0.841
Sampling Chain 13...
Accuracy of cell classification on all data: 0.797
Sa

Classify cells based on accepted MP trees, and write predictions to file...

In [5]:
burnt_samples = [sample for chain in accepted_MP for sample in chain[-1:]]
Y_predict = classify_cells_majority(data, burnt_samples, table, ct2idx)
accuracy = sum(Y == Y_predict)*1.0/ N
print "Accuracy of cell classification: %.3f" % (accuracy)

df['MP_prediction'] = pd.Series([idx2ct[i] for i in Y_predict], index=df.index)
df.to_csv(FILENAME_PREDICTIONS, compression='gzip', index = False)

Accuracy of cell classification: 0.923


In [6]:
print [len(i) for i in accepted_MP]

[100, 49, 99, 64, 79, 98, 86, 67, 61, 102, 79, 41, 82, 63, 70, 83, 103, 75, 95, 77, 72, 55, 49, 77, 70, 76, 96, 89, 53, 108, 92, 71, 43, 68, 84, 53, 63, 89, 73, 33, 112, 53, 81, 71, 71, 70, 55, 69, 77, 72]


In [9]:
print table.shape

(19, 13)
